In [1]:
import torch
from torch import nn

from IPython.display import display as d

In [2]:
ckpt = torch.load("logs/train/runs/2024-07-15_01-48-43/checkpoints/epoch_009.ckpt")

In [4]:
d(ckpt["state_dict"])

OrderedDict([('net.layers.0.branched_linear_block_xyz.0.0.weight',
              tensor([[ 0.6286],
                      [ 0.0729],
                      [ 0.7486],
                      [ 0.8422],
                      [-0.9058],
                      [ 0.3972],
                      [-0.9412],
                      [-0.4458],
                      [-0.1331],
                      [-0.3629],
                      [ 0.9026],
                      [-0.7108],
                      [-0.5556],
                      [ 0.8622],
                      [-0.7945],
                      [ 0.0651],
                      [-1.2834],
                      [-0.2022],
                      [-0.1035],
                      [ 0.0921],
                      [ 0.5901],
                      [ 1.0616],
                      [-0.0546],
                      [-1.1460],
                      [ 1.0872],
                      [-0.8884],
                      [ 0.6542],
                      [-0.1369],
         

In [ ]:
from src.models.pde_lit_model_xt import PDELitModule

In [ ]:
class PretrainedModel(nn.Module):
    def __init__(self, ckpt_path: str):
        super().__init__()
        
        self.net = PDELitModule.load_from_checkpoint(ckpt_path)
        self.net.eval()
        self.net.freeze()

    def forward(self, inputs):
        return self.net(inputs)

In [ ]:
pretrained_model = PretrainedModel("logs/train/runs/2024-07-15_01-48-43/checkpoints/epoch_009.ckpt")

/home/danil/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
